### Crisp County Georgia 2022 Municipal Election Returns Joined to Precinct Boundaries

#### Sources
Precint-level data from [Georgia Secretary of State Certified Results by County - XML format](https://results.enr.clarityelections.com/GA/114891/web.285569/#/reporting).

RDH Statewide General Election Results and Precinct Boundaries[ Georgia 2022 General Election Precinct-Level Results and Boundaries](https://redistrictingdatahub.org/dataset/georgia-2022-general-election-precinct-level-results-and-boundaries/)
    
County-level data to run checks from [Georgia Secretary of State County Summary Report - XML format](https://results.enr.clarityelections.com//GA//114891/300218/reports/detailxml.zip)

In [1]:
import geopandas as gp
import pandas as pd
import os
pd.set_option('display.max_rows', None)

<p><a name="ETL"></a></p>

#### Read in Precint Level Election Results

In [2]:
# Read in precint level election results
er = pd.read_csv("./raw-from-source/GA22_CRISP_ER.csv")

In [3]:
er.head(1)

,UNIQUE_ID,MNCIPAL,PRECINCT,GCNC2RDOW,GBOE1RBRI,GBOE1DFOR,GBOE2DPOS,GBOE2RADK,GBOE3DGRA,GBOE3RHAM,GBOE4RAND,GBOE5RWRI,GBOE4DWIL
0,Crisp-:-Arabi,Crisp,Arabi,391,241,102,0,0,0,0,75,348,87


#### Read in 2022 General Statewide Shapefile

In [4]:
ga_gen_shp = gp.read_file('./raw-from-source/ga_2022_gen_prec/ga_2022_gen_prec_no_splits/ga_2022_gen_prec_no_splits.shp')
# make sure unique id for precinct match lines up
ga_gen_shp['UNIQUE_ID'] = ga_gen_shp['county'] +'-:-'+ ga_gen_shp['precinct']
# limit to Crisp County
crisp_shp = ga_gen_shp[ga_gen_shp['county'] == 'Crisp']

In [5]:
#Check for precinct names, looking for 5
crisp_shp.precinct

745        Arabi
746        Coney
747      Cordele
748    Jamestown
749     Listonia
Name: precinct, dtype: object

#### Merge with Election Returns

In [6]:
# merge
crisp_pber = crisp_shp[['UNIQUE_ID', 'geometry']].merge(er, on='UNIQUE_ID', how='outer', indicator=True)

In [7]:
# check
crisp_pber._merge.value_counts()

both          5
left_only     0
right_only    0
Name: _merge, dtype: int64

In [8]:
#Drop Merge Column
crisp_pber = crisp_pber.drop(columns=['_merge'])

In [10]:
# rearrange columns
crisp_pber = crisp_pber[['UNIQUE_ID', 'MNCIPAL','PRECINCT'] + crisp_pber.columns[4:].to_list() + ['geometry']]

In [11]:
# Clean up
crisp_pber['geometry'] = crisp_pber.geometry.buffer(0)
#Fill NA
crisp_pber = crisp_pber.fillna(0)

#### Check Vote Totals against SOS totals

In [12]:
# Read in SOS totals
#Read in statewide csv summary file, with county level data
sos = pd.read_csv("./raw-from-source/summary/summary.csv")
# look at contest names, and subset to contests of interest only
#sos['contest name']
sos = sos[sos['contest name'].isin(['County Comm Dist 2 (Vote For 1)', 'County BOE Dist 1 (Vote For 1)', 'County BOE Dist 2 (Vote For 1)', 'County BOE Dist 3 (Vote For 1)', 'County BOE Dist 4 (Vote For 1)', 'County BOE Dist 5 LRG (Vote For 1)'])]

In [13]:
sos

,line number,contest name,choice name,party name,total votes,percent of votes,registered voters,ballots cast,num Precinct total,num Precinct rptg,over votes,under votes
30,31,County Comm Dist 2 (Vote For 1),"James R. """"Dickie"""" Dowdy (I) (Rep)",NaN,4446,100.00,12470,6356,5,5,0,508
31,32,County BOE Dist 1 (Vote For 1),Jay L. Brinson (Rep),NaN,1318,67.45,10382,4829,3,3,0,20
32,33,County BOE Dist 1 (Vote For 1),Scott Forehand (Dem),NaN,636,32.55,10382,4829,3,3,0,20
33,34,County BOE Dist 2 (Vote For 1),Lydia Dent Adkins (Rep),NaN,361,40.25,8751,3884,1,1,0,27
34,35,County BOE Dist 2 (Vote For 1),Crandall O. Postell (Dem),NaN,536,59.75,8751,3884,1,1,0,27
35,36,County BOE Dist 3 (Vote For 1),Whitnye Hamilton (Rep),NaN,591,46.50,8751,3884,1,1,0,33
36,37,County BOE Dist 3 (Vote For 1),Anna Granville (Dem),NaN,680,53.50,8751,3884,1,1,0,33
37,38,County BOE Dist 4 (Vote For 1),Chris Anderson (Rep),NaN,1832,100.00,11641,5854,4,4,0,290
38,39,County BOE Dist 5 LRG (Vote For 1),Joe Joe Wright (Rep),NaN,4201,67.25,12470,6356,5,5,0,106
39,40,County BOE Dist 5 LRG (Vote For 1),Katrisha Williams (Dem),NaN,2046,32.75,12470,6356,5,5,0,106


In [14]:
crisp_pber.sum()

UNIQUE_ID    Crisp-:-ArabiCrisp-:-ConeyCrisp-:-CordeleCrisp...
MNCIPAL                              CrispCrispCrispCrispCrisp
PRECINCT                    ArabiConeyCordeleJamestownListonia
GCNC2RDOW                                                 4446
GBOE1RBRI                                                 1318
GBOE1DFOR                                                  636
GBOE2DPOS                                                  536
GBOE2RADK                                                  361
GBOE3DGRA                                                  680
GBOE3RHAM                                                  591
GBOE4RAND                                                 1832
GBOE5RWRI                                                 4201
GBOE4DWIL                                                 2046
dtype: object

In [15]:
#final visual check
crisp_pber.head()

,UNIQUE_ID,MNCIPAL,PRECINCT,GCNC2RDOW,GBOE1RBRI,GBOE1DFOR,GBOE2DPOS,GBOE2RADK,GBOE3DGRA,GBOE3RHAM,GBOE4RAND,GBOE5RWRI,GBOE4DWIL,geometry
0,Crisp-:-Arabi,Crisp,Arabi,391,241,102,0,0,0,0,75,348,87,"POLYGON ((-83.61349 31.80411, -83.62476 31.803..."
1,Crisp-:-Coney,Crisp,Coney,625,708,405,0,0,0,0,626,595,66,"POLYGON ((-83.84050 32.03019, -83.84047 32.030..."
2,Crisp-:-Cordele,Crisp,Cordele,2132,369,129,536,361,680,591,319,2027,1783,"POLYGON ((-83.76702 31.90110, -83.77568 31.901..."
3,Crisp-:-Jamestown,Crisp,Jamestown,813,0,0,0,0,0,0,812,763,79,"POLYGON ((-83.85766 31.90203, -83.85637 31.902..."
4,Crisp-:-Listonia,Crisp,Listonia,485,0,0,0,0,0,0,0,468,31,"POLYGON ((-83.61190 31.89937, -83.61330 31.899..."


<p><a name="exp"></a></p>

### Export Joined Precinct Level Dataset

In [16]:
if not os.path.exists("./GA22_CRISP_PBER/"):
    os.mkdir("./GA22_CRISP_PBER/")
    
crisp_pber.to_file("./GA22_CRISP_PBER/GA22_CRISP_PBER.shp")